# Penghindaran Tabrakan - Pengumpulan Data

Jika kita menjalankan program `gerak dasar`,  kita menikmati betapa mudahnya membuat Jetbot bergerak! Itu sangat keren! Tapi yang lebih keren lagi, membuat JetBot bergerak sendiri!

Ini adalah tugas yang sangat sulit, yang memiliki banyak pendekatan, tetapi seluruh masalah biasanya dipecah menjadi sub masalah yang lebih mudah. Dapat dikatakan bahwa salah satu sub masalah yang paling penting untuk dipecahkan, adalah masalah mencegah robot memasuki situasi berbahaya! kita menyebutnya *penghindaran tabrakan*.

Dalam materi kali ini, kita akan mencoba memecahkan masalah menggunakan pembelajaran mendalam atau yang biasa disebut *Deep Learning* dan satu sensor yang sangat serbaguna: kamera. kita akan melihat bagaimana dengan jaringan saraf (Neural Network), kamera, dan NVIDIA Jetson Nano, kami dapat mengajari robot perilaku yang sangat berguna!

Pendekatan yang kita ambil untuk menghindari tabrakan adalah dengan membuat "safety bubble" virtual di sekitar robot. Dalam pengaman ini, robot dapat berputar dalam lingkaran tanpa menabrak benda apa pun (atau situasi berbahaya lainnya seperti jatuh dari jalur).


Tentu saja, robot dibatasi oleh apa yang ada di bidang penglihatannya, dan kita tidak dapat mencegah objek ditempatkan di belakang robot, dll. Tetapi kita dapat mencegah robot memasuki skenario yang kita buat.


Cara yang akan kita lakukan ini sangat sederhana:

Pertama, kita akan menempatkan robot secara manual dalam skenario yang akan kita buat di mana "safety bubble" dilanggar, dan memberi label skenario ini ``blocked``. kita akan menyimpan cuplikan (gambar) dari apa yang dilihat robot bersama dengan label ini.

Kedua, kita akan menempatkan robot secara manual dalam skenario yang aman untuk bergerak maju sedikit, dan memberi label skenario ini ``free``. Demikian juga, kami menyimpan snapshot bersama dengan label ini.

Itu saja yang akan kita lakukan dalam materi kali ini; pengumpulan data. Setelah kita memiliki banyak gambar dan label, kami akan mengunggah data ini ke mesin berkemampuan GPU tempat kami akan *melatih* jaringan saraf untuk memprediksi.

> CATATAN PENTING: Saat JetBot berputar di tempatnya, ia sebenarnya berputar di sekitar pusat di antara kedua roda, bukan di tengah sasis robot itu sendiri. Ini adalah detail penting yang perlu diingat ketika kita mencoba memperkirakan apakah "safety bubble" robot dilanggar atau tidak. Tapi jangan khawatir, Anda tidak harus tepat. Jika ragu, lebih baik bersandar pada sisi yang berhati-hati (gelembung keamanan besar). Kita ingin memastikan JetBot tidak memasuki skenario yang tidak dapat dihindari dengan memutar di tempatnya.

### Menampilkan kamera secara langsung

Jadi mari kita mulai. Pertama, mari kita inisialisasi dan menampilkan kamera kita seperti yang kita lakukan pada materi *teleoperasi*.

> Jaringan saraf kita mengambil gambar 224x224 piksel sebagai masukan. kita akan mengatur kamera ke ukuran itu untuk meminimalkan ukuran file dari kumpulan data.
> Dalam beberapa skenario, mungkin lebih baik untuk mengumpulkan data dalam ukuran gambar yang lebih besar dan menurunkan skala ke ukuran yang diinginkan nanti.

In [2]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Luar biasa, selanjutnya mari kita buat beberapa direktori tempat kita akan menyimpan semua data kita. kita akan membuat folder ``dataset`` yang akan berisi dua sub-folder ``free`` dan ``blocked``,
di mana kita akan menempatkan gambar untuk setiap skenario.

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# kita memiliki pernyataan "try/expect" ini karena fungsi berikut dapat menimbulkan kesalahan jika direktori sudah ada

try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


Jika Anda me-refresh browser file Jupyter di sebelah kiri, Anda sekarang akan melihat direktori tersebut muncul. Selanjutnya, mari kita buat dan tampilkan beberapa tombol yang akan kita gunakan untuk menyimpan snapshot
untuk setiap label kelas. Kita juga akan menambahkan beberapa kotak teks yang akan menampilkan berapa banyak gambar dari setiap kategori yang telah kami kumpulkan sejauh ini. Ini berguna karena kami ingin membuat

tentu saja kita mengumpulkan gambar ``free`` sebanyak gambar ``bloked``. Ini juga membantu untuk mengetahui berapa banyak gambar yang telah kami kumpulkan secara keseluruhan.

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Saat ini, tombol-tombol ini tidak akan melakukan apa-apa. Kita harus melampirkan fungsi untuk menyimpan gambar untuk setiap kategori ke fungsi ``on_click`` tombol. lalu kita akan menyimpan nilainya
widget ``Image``,  sudah dalam format JPEG terkompresi!

Untuk memastikan kita tidak mengulangi nama file yang sama (bahkan di mesin yang berbeda!), kami akan menggunakan paket ``uuid`` dalam python, yang mendefinisikan metode ``uuid1`` untuk menghasilkan pengenal unik. Pengidentifikasi unik ini dihasilkan dari informasi seperti waktu saat ini dan alamat mesin.

In [5]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# lampirkan "callback", kita menggunakan fungsi 'lambda'.
# parameter yang akan diberikan oleh acara on_click ke fungsi kita

free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Sekarang tombol di atas akan menyimpan gambar ke direktori ``free`` dan ``blocked``. Kita dapat menggunakan browser file Jupyter Lab untuk melihat file-file ini!

Sekarang lanjutkan dan kumpulkan beberapa data

1. Tempatkan robot dalam skenario di mana robot diblokir dan tekan ``tambahkan yang diblokir``
2. Tempatkan robot dalam skenario gratis dan tekan ``add free``
3. Ulangi 1, 2

> PENGINGAT: Kita dapat memindahkan widget ke jendela baru dengan mengklik kanan sel dan mengklik ``Buat Tampilan Baru untuk Output``. Atau, Anda bisa menampilkannya kembali
> bersama-sama seperti yang akan kita lakukan di bawah ini

Berikut adalah beberapa tip untuk melabeli data

1. Coba orientasi yang berbeda
2. Coba pencahayaan yang berbeda
3. Cobalah berbagai jenis objek/tabrakan; dinding, tepian, benda
4. Coba lantai / benda bertekstur yang berbeda; bermotif, halus, kaca, dll.

Pada akhirnya, semakin banyak data yang kita miliki tentang skenario yang akan dihadapi robot di dunia nyata, semakin baik perilaku menghindari tabrakan kita. Ini penting
untuk mendapatkan *bervariasi* data (seperti yang dijelaskan oleh tip di atas) dan bukan hanya banyak data, tetapi Anda mungkin memerlukan setidaknya 100 gambar dari setiap kelas (itu bukan sains, hanya tip yang berguna di sini). Tapi jangan khawatir, itu berjalan cukup cepat begitu Anda mulai :)

In [6]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
camera.stop()